# Core10_05a — Tree Policy Evaluation (Leaf-as-Policy)

목적:
- Decision Tree를 확률 예측기가 아니라 **정책 분기기(policy router)**로 사용
- threshold sweep 제거
- leaf_id → POLICY → COST 구조로 평가
- Option A / Option B pseudo_failure 계약을 병렬 비교

핵심 원칙:
- ❌ leaf → hazard → threshold
- ✅ leaf → POLICY CLASS → COST

In [22]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

IN_PATH = Path("../artifact/core10/core10_04_survivability_scores.csv")
assert IN_PATH.exists(), f"Input not found: {IN_PATH.resolve()}"

df = pd.read_csv(IN_PATH)

print("rows:", len(df))
print("cols:", df.columns.tolist())

rows: 19
cols: ['antibody_id', 'signature', 'core10_operational_risk', 'proxy_survivability_score', 'tie_break_risk', 'logit_safe_prob', 'hazard_logit', 'tree_safe_prob', 'hazard_tree', 'hc_subtype', 'lc_subtype', 'cluster_size']


In [23]:
# Rule hazard
df["hazard_rule"] = (1.0 - df["proxy_survivability_score"]).clip(0, 1)

# Composite hazard (Rule + Tie-break)
df["composite_hazard"] = (
    0.7 * df["hazard_rule"] +
    0.3 * df["tie_break_risk"]
).clip(0, 1)

df[[
    "antibody_id",
    "hazard_rule",
    "tie_break_risk",
    "composite_hazard"
]].head(10)

,antibody_id,hazard_rule,tie_break_risk,composite_hazard
0,GDPa1-060,0.141005,1.308744,0.491327
1,GDPa1-085,0.155394,1.234429,0.479104
2,GDPa1-021,0.155394,1.244671,0.482177
3,GDPa1-025,0.155394,1.335000,0.509276
4,GDPa1-165,0.155394,1.350459,0.513913
5,GDPa1-017,0.158631,1.154242,0.457314
6,GDPa1-010,0.165528,1.336067,0.516690
7,GDPa1-138,0.494605,1.335473,0.746865
8,GDPa1-050,0.508993,1.426338,0.784197
9,GDPa1-039,0.508993,1.510580,0.809469


In [24]:
PSEUDO_FAIL_QUANTILE = 0.30

# Option A: fallback pool 내부 상대 위험
fail_th_A = df["hazard_rule"].quantile(1.0 - PSEUDO_FAIL_QUANTILE)
df["pseudo_failure_A"] = (df["hazard_rule"] >= fail_th_A).astype(int)

# Option B: composite hazard 기준
fail_th_B = df["composite_hazard"].quantile(1.0 - PSEUDO_FAIL_QUANTILE)
df["pseudo_failure_B"] = (df["composite_hazard"] >= fail_th_B).astype(int)

print("Option A threshold:", float(fail_th_A))
print(df["pseudo_failure_A"].value_counts())

print("Option B threshold:", float(fail_th_B))
print(df["pseudo_failure_B"].value_counts())

Option A threshold: 0.7329374671571203
pseudo_failure_A
0    13
1     6
Name: count, dtype: int64
Option B threshold: 0.9237876944869656
pseudo_failure_B
0    13
1     6
Name: count, dtype: int64


In [25]:
from sklearn.tree import DecisionTreeClassifier

FEATURES = [
    "core10_operational_risk",
    "tie_break_risk",
    "cluster_size",
]

X = df[FEATURES].fillna(0.0)
n = len(df)

def train_tree(y):
    tree = DecisionTreeClassifier(
        max_depth=2,
        min_samples_leaf=int(0.15 * n),
        min_samples_split=int(0.30 * n),
        class_weight="balanced",
        random_state=42
    )
    tree.fit(X, y)
    return tree

tree_A = train_tree(df["pseudo_failure_A"])
tree_B = train_tree(df["pseudo_failure_B"])

print("Tree A:", {
    "depth": tree_A.get_depth(),
    "n_leaves": tree_A.get_n_leaves()
})
print("Tree B:", {
    "depth": tree_B.get_depth(),
    "n_leaves": tree_B.get_n_leaves()
}) # Tree 학습 (정책 분기 전용)

Tree A: {'depth': 2, 'n_leaves': 3}
Tree B: {'depth': 2, 'n_leaves': 3}


In [26]:
def leaf_stats(df, tree, label_col, prefix):
    leaf_id = tree.apply(X)
    df[f"leaf_id_{prefix}"] = leaf_id

    stats = (
        df.groupby(f"leaf_id_{prefix}")[label_col]
          .agg(n="size", fail_rate="mean")
          .reset_index()
    )
    return stats

leaf_stats_A = leaf_stats(df, tree_A, "pseudo_failure_A", "A")
leaf_stats_B = leaf_stats(df, tree_B, "pseudo_failure_B", "B")

leaf_stats_A, leaf_stats_B # Leaf -> 정책통계 (threhold 없음)

(   leaf_id_A   n  fail_rate
 0          1  13        0.0
 1          3   2        1.0
 2          4   4        1.0,
    leaf_id_B   n  fail_rate
 0          1  13        0.0
 1          3   2        1.0
 2          4   4        1.0)

In [27]:
# 정책 정의: leaf는 위험도가 아니라 "행동"
POLICY_MAP = {
    "SAFE": 0,    # 그대로 운영
    "WATCH": 2,   # 모니터링 비용
    "DROP": 8     # 교체/중단 비용
}

def assign_policy(fail_rate):
    if fail_rate < 0.25:
        return "SAFE"
    elif fail_rate < 0.75:
        return "WATCH"
    else:
        return "DROP"

In [28]:
leaf_stats_A["policy"] = leaf_stats_A["fail_rate"].map(assign_policy)
leaf_stats_A["policy_cost"] = leaf_stats_A["policy"].map(POLICY_MAP)

policy_map_A = dict(
    zip(leaf_stats_A["leaf_id_A"], leaf_stats_A["policy"])
)
cost_map_A = dict(
    zip(leaf_stats_A["leaf_id_A"], leaf_stats_A["policy_cost"])
)

df["policy_A"] = df["leaf_id_A"].map(policy_map_A)
df["policy_cost_A"] = df["leaf_id_A"].map(cost_map_A)

print("Total cost (Option A):", df["policy_cost_A"].sum())
df[["antibody_id","leaf_id_A","policy_A","policy_cost_A"]].head(10) # Leaf -> POLICY -> COST (Option A)

Total cost (Option A): 48


,antibody_id,leaf_id_A,policy_A,policy_cost_A
0,GDPa1-060,1,SAFE,0
1,GDPa1-085,1,SAFE,0
2,GDPa1-021,1,SAFE,0
3,GDPa1-025,1,SAFE,0
4,GDPa1-165,1,SAFE,0
5,GDPa1-017,1,SAFE,0
6,GDPa1-010,1,SAFE,0
7,GDPa1-138,1,SAFE,0
8,GDPa1-050,1,SAFE,0
9,GDPa1-039,1,SAFE,0


In [29]:
leaf_stats_B["policy"] = leaf_stats_B["fail_rate"].map(assign_policy)
leaf_stats_B["policy_cost"] = leaf_stats_B["policy"].map(POLICY_MAP)

policy_map_B = dict(
    zip(leaf_stats_B["leaf_id_B"], leaf_stats_B["policy"])
)
cost_map_B = dict(
    zip(leaf_stats_B["leaf_id_B"], leaf_stats_B["policy_cost"])
)

df["policy_B"] = df["leaf_id_B"].map(policy_map_B)
df["policy_cost_B"] = df["leaf_id_B"].map(cost_map_B)

print("Total cost (Option B):", df["policy_cost_B"].sum())
df[["antibody_id","leaf_id_B","policy_B","policy_cost_B"]].head(10) # Leaf → POLICY → COST (Option B)

Total cost (Option B): 48


,antibody_id,leaf_id_B,policy_B,policy_cost_B
0,GDPa1-060,1,SAFE,0
1,GDPa1-085,1,SAFE,0
2,GDPa1-021,1,SAFE,0
3,GDPa1-025,1,SAFE,0
4,GDPa1-165,1,SAFE,0
5,GDPa1-017,1,SAFE,0
6,GDPa1-010,1,SAFE,0
7,GDPa1-138,1,SAFE,0
8,GDPa1-050,1,SAFE,0
9,GDPa1-039,1,SAFE,0


In [30]:
summary = pd.DataFrame({
    "option": ["A", "B"],
    "total_cost": [
        df["policy_cost_A"].sum(),
        df["policy_cost_B"].sum()
    ],
    "n_SAFE": [
        (df["policy_A"] == "SAFE").sum(),
        (df["policy_B"] == "SAFE").sum()
    ],
    "n_WATCH": [
        (df["policy_A"] == "WATCH").sum(),
        (df["policy_B"] == "WATCH").sum()
    ],
    "n_DROP": [
        (df["policy_A"] == "DROP").sum(),
        (df["policy_B"] == "DROP").sum()
    ],
})

summary

,option,total_cost,n_SAFE,n_WATCH,n_DROP
0,A,48,13,0,6
1,B,48,13,0,6


Under the current fallback pool distribution and conservative tree constraints
(max_depth=2, large leaf sizes),
pseudo-failure definitions based on rule-only hazard
and rule+tie-break composite hazard
resulted in identical leaf-level policy assignments and operational costs.
This indicates that the additional tie-break signal does not alter
policy-level decisions at the current operational granularity.